In [46]:
import pandas as pd
from sqlalchemy import create_engine

In [52]:
db_params = {
    "dbname": "library",
    "user": "postgres",
    "password": "4473",
    "host": "localhost",
    "port": "5432"  
}

engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")


In [210]:
#Staging
book_copy_df=pd.read_sql('select * from bookcopy',engine)
book_df=pd.read_sql('select * from book',engine)
employee_df=pd.read_sql('select * from employee',engine)
fine_df=pd.read_sql('select * from fine',engine)
loan_df=pd.read_sql('select * from loan',engine)
member_df=pd.read_sql('select * from member',engine)
position_df=pd.read_sql('select * from position',engine)
provider_df=pd.read_sql('select * from provider',engine)

In [222]:
loan_df['loanstart'] = pd.to_datetime(df['loanstart'])
loan_df['loanend'] = pd.to_datetime(df['loanend'])

In [224]:
#Transform
agg_member_loans_df=loan_df.groupby('memberid').agg(
    number_of_loans=('loanid', 'count'),
    last_loan_date=('loanstart', 'max')).reset_index()


merged_df = fine_df.merge(loan_df, on='loanid')
agg_fines_df=merged_df.groupby('memberid').agg(
    total_fines_values=('amount', 'sum'),
    unpaid_fines_values=('amount', lambda x: x[merged_df.loc[x.index, 'ispaid'] == False].sum())
).reset_index()


agg_book_availability_df=book_copy_df.groupby('status').agg(
    total_books=('copyid','count')).reset_index()

df = loan_df.merge(book_copy_df, on='copyid', how='inner')
df = df.merge(book_df, on='iban', how='inner')
df = df.merge(fine_df, on='loanid', how='left')

df = df.assign(
    days_borrowed=(df['loanend'] - df['loanstart']).dt.days,
    fine_amount=df['amount'].fillna(0),
    isPaid=df['ispaid'].fillna(True).astype(bool)
)

dw_loans_summary=df[['loanid', 'memberid', 'title', 'loanstart', 'loanend', 'days_borrowed', 'fine_amount', 'isPaid']]

C:\Users\butea\AppData\Local\Temp\ipykernel_23600\2989250371.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  isPaid=df['ispaid'].fillna(True).astype(bool)


In [226]:
#Load
agg_member_loans_df.to_sql('agg_member_loans', engine, if_exists='replace', index=False)
agg_fines_df.to_sql('agg_fines', engine, if_exists='replace', index=False)
agg_book_availability_df.to_sql('agg_book_availability', engine, if_exists='replace', index=False)
dw_loans_summary.to_sql('dw_loans_summary', engine, if_exists='replace', index=False)

568